In [1]:
# Instalar Dash y dependencias
!pip install pandas dash plotly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 57.5 MB/s eta 0:00:00


In [2]:
!wget -O spacex_launch_dash.csv "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"


--2025-09-30 18:37:53--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2025-09-30 18:37:53 (551 MB/s) - ‘spacex_launch_dash.csv’ saved [2476/2476]



In [3]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

# Cargar datos
spacex_df = pd.read_csv("spacex_launch_dash.csv")

# Rango de payload
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


In [4]:
# Crear aplicación Dash
app = dash.Dash(__name__)

# Layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard', style={'textAlign': 'center'}),

    # TASK 1: Dropdown para Launch Site
    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label': 'All Sites', 'value': 'ALL'}] +
                [{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()],
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),

    html.Br(),

    # TASK 2: Pie chart
    html.Div(dcc.Graph(id='success-pie-chart')),

    html.Br(),

    # TASK 3: Range slider para Payload
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={0: '0', 10000: '10000'},
        value=[min_payload, max_payload]
    ),

    html.Br(),

    # TASK 4: Scatter plot
    html.Div(dcc.Graph(id='success-payload-scatter-chart'))
])


In [5]:
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(
            spacex_df,
            values='class',
            names='Launch Site',
            title='Total Success Launches by Site'
        )
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(
            filtered_df,
            names='class',
            title=f'Total Success Launches for site {entered_site}'
        )
        return fig


In [6]:
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [
        Input('site-dropdown', 'value'),
        Input('payload-slider', 'value')
    ]
)
def update_scatter_chart(entered_site, payload_range):
    low, high = payload_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) &
                            (spacex_df['Payload Mass (kg)'] <= high)]

    if entered_site == 'ALL':
        fig = px.scatter(
            filtered_df,
            x="Payload Mass (kg)",
            y="class",
            color="Booster Version Category",
            title="Correlation between Payload and Success for All Sites"
        )
        return fig
    else:
        site_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(
            site_df,
            x="Payload Mass (kg)",
            y="class",
            color="Booster Version Category",
            title=f"Correlation between Payload and Success for site {entered_site}"
        )
        return fig


In [7]:
!pip install pyngrok
from pyngrok import ngrok

# Asignar puerto
port = 8050
public_url = ngrok.connect(port)
print(public_url)

# Ejecutar app
app.run_server(port=port)


ERROR:pyngrok.process.ngrok:t=2025-09-30T18:38:54+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-09-30T18:38:54+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-09-30T18:38:54+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [8]:
fig_pie_all = px.pie(
    spacex_df,
    values='class',
    names='Launch Site',
    title='Total Success Launches by Site'
)
fig_pie_all.show()



In [9]:
# Filtrar solo KSC
ksc_df = spacex_df[spacex_df['Launch Site'] == 'KSC LC-39A']

# Pie chart de éxitos vs fracasos
fig_ksc = px.pie(
    ksc_df,
    names='class',
    title='Launch Success vs Failure for KSC',
    color='class',
    color_discrete_map={0: 'red', 1: 'green'}
)

fig_ksc.show()


In [10]:
import pandas as pd
import plotly.express as px

# Cargar datos
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

# Definir rangos de payload para simular el slider
payload_ranges = [
    (0, 2000),
    (2000, 4000),
    (4000, 6000),
    (6000, 8000),
    (8000, 10000)
]

# Generar y mostrar scatter plots para cada rango
for low, high in payload_ranges:
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) &
                             (spacex_df['Payload Mass (kg)'] <= high)]

    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        title=f'Payload vs Launch Outcome for all sites ({low}-{high} kg)'
    )
    fig.show()


In [11]:
import pandas as pd
import plotly.express as px

# Cargar datos
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

# Definir rangos de payload
def payload_range_label(payload):
    if payload < 2000:
        return '0-2k'
    elif payload < 4000:
        return '2k-4k'
    elif payload < 6000:
        return '4k-6k'
    elif payload < 8000:
        return '6k-8k'
    else:
        return '8k-10k'

spacex_df['Payload Range'] = spacex_df['Payload Mass (kg)'].apply(payload_range_label)

# Scatter plot único
fig = px.scatter(
    spacex_df,
    x='Payload Mass (kg)',
    y='class',
    color='Booster Version Category',  # color por booster
    symbol='Payload Range',            # forma por rango de payload
    title='Payload vs Launch Outcome (all sites) with Booster and Payload Range',
    labels={'class': 'Launch Outcome'}
)

fig.show()
